# LB03b Deep Dreaming
<img src="resources/LB03b_Deep_Dreaming_SBG.png" style="height:350px;"/>

Use the given code to modify a picture of yourself. Deep dreaming can help you visualize how a convolutional neural network interprets a picture.

In [52]:
import os

import numpy as np
from imageio import imwrite
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg16
from keras import backend as K
from keras.layers import Input
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [53]:
# util function to open, resize and format picturs into appropriate tensors.
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

In [54]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    x = x.reshape((img_width, img_height, 3))

    # remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] ++ 123.68

    # BGR -> RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [55]:
# continuity loss util function
def continuity_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :img_width - 1, :img_height-1, :] -
                 x[:, 1:, :img_height - 1, :])
    b = K.square(x[:, :img_width - 1, :img_height-1, :] -
                 x[:, :img_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [56]:
# this Evaluator class makes it possible to compute loss and gradients in one pass while retrieving them via two
# separate functions, "loss" and "grads". This is done because scipy.optimize requires separate functions for
# loss and gradients, but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

## LB03a.1 Specify a path to your image

* Additionally, specify a prefix that is prepended to your output images.
* Also, don't forget to actually upload an image!

In [57]:
img_dir = 'images/'

base_image_path = img_dir + 'pic_to_load.jpg'
result_prefix = img_dir + 'out_'

if not os.path.exists(img_dir):
    os.makedirs(img_dir)

## LB03a.2 Change settings

* You may play around with the settings specified in this cell to alter the output image.

In [58]:
# maximum of iterations to use, the more the stronger the effect
max_iterations= 20

# dimensions of the generated picture
img_width = 470
img_height = 470

# some settings we found interesting
saved_settings = {
    'bad_trip': {'features':{'block4_conv1': 0.05,
                            'block4_conv2': 0.01,
                            'block4_conv3': 0.01},
                'continuity': 0.01,
                'dream_l2': 0.8,
                'jitter': 5},
    'dreamy': {'features': {'block5_conv1':0.05,
                            'block5_conv2': 0.02},
                'continuity': 0.1,
                'dream_l2': 0.02,
                'jitter': 0},
    }

# define the settings to use
settings = saved_settings['dreamy']

## LB03a.3 Start dreaming!

* Rerun this cell as often as you'd like with as many different images as you want.

In [51]:
img_size = (img_width, img_height, 3)

# this contains the generated image
dream = Input(batch_shape=(1,) + img_size)

# build the VGG16 network with the input placeholder
# the model will be loaded with pre-trained imagenet weights
model = vgg16.VGG16(input_tensor=dream, weights='imagenet', include_top=False)

# get symbolic outputs of each "key" layer (they have all unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# define the loss
loss = K.variable(0.)
for layer_name in settings['features']:
    # add the L2 norm of the features of a layer to the loss
    assert layer_name in layer_dict.keys(), 'Layer ' + layer_name + ' not found in model.'
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    shape = layer_dict[layer_name].output_shape
    # avoid border artifacts by only involving non-border pixels in the loss
    loss = loss - coeff * K.sum(K.square(x[:, 2: shape[1] - 2, 2: shape[2] - 2, :])) / np.prod(shape[1:])

# add continuity loss (gives image local coherence, can result in an artful blur)
loss += settings['continuity'] * continuity_loss(dream) / np.prod(img_size)
# add image L2 norm to loss (prevents pixels from taking very high values, makes image darker)
loss += settings['dream_l2'] * K.sum(K.square(dream)) / np.prod(img_size)

# feel free to further modify the loss as you see fit, to achieve new effects...

# compute the gradients of the dream wrt the loss
grads = K.gradients(loss, dream)

outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([dream], outputs)
def eval_loss_and_grads(x):
    x = x.reshape((1,) + img_size)
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values


evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image, so as to minimize the loss
x = preprocess_image(base_image_path)
#x=np.random.randn(1,img_width, img_height,3)
for i in range(max_iterations):
    print('Start of iteration', i)
    start_time = time.time()

    # add a random jitter to the initial image. this will be reverted at decoding time
    random_jitter = (settings['jitter'] * 2) * (np.random.random(img_size) - 0.5)
    x += random_jitter

    # run L-BFGS for 7 steps
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=7)
    print('Current loss value:', min_val)
    # decode the dream and save it
    x = x.reshape(img_size)
    x -= random_jitter
    img = deprocess_image(np.copy(x))
    fname = result_prefix + '_at_iteration_%02d.png' % i
    imwrite(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: -71.25282
Image saved as images/out__at_iteration_0.png
Iteration 0 completed in 8s
Start of iteration 1
Current loss value: -231.8913
Image saved as images/out__at_iteration_1.png
Iteration 1 completed in 8s
Start of iteration 2
Current loss value: -521.01746
Image saved as images/out__at_iteration_2.png
Iteration 2 completed in 8s
Start of iteration 3
Current loss value: -916.5678
Image saved as images/out__at_iteration_3.png
Iteration 3 completed in 8s
Start of iteration 4
Current loss value: -1452.6953
Image saved as images/out__at_iteration_4.png
Iteration 4 completed in 7s
Start of iteration 5
Current loss value: -2172.5498
Image saved as images/out__at_iteration_5.png
Iteration 5 completed in 7s
Start of iteration 6
Current loss value: -3132.8433
Image saved as images/out__at_iteration_6.png
Iteration 6 completed in 7s
Start of iteration 7
Current loss value: -4413.435
Image saved as images/out__at_iteration_7.png
Iteration 7 completed in